<a href="https://colab.research.google.com/github/profteachkids/CHE3022_Fall2022/blob/main/NetworkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import root

In [2]:
mu = 1e-3  #Pa s
rho = 1e3   # kg/m3
g=9.81      #m/s2

In [3]:
def churchill(Re, eD):
    t1 = (-2.457*np.log((7./Re)**0.9 + 0.27*eD))**16
    t2 = (37530/Re)**16
    return 2* ((8/Re)**12 + 1/(t1+t2)**1.5)**(1/12.)

In [4]:
def headDrop(Q, L, D, eD, K=0.):
   v=4*np.abs(Q)/(np.pi*(D**2))
   Re = rho * v * D / mu
   f = churchill(Re, eD)
   return np.copysign(2*f*L*(v**2)/(g*D) + K/2/g*(v**2),Q)

In [72]:
#hD is the decrease in head from inlet to outlet
#Positive hD indicates positive from inlet to outlet
def flow(hD, L, D, eD, K=0.):

    if np.abs(hD)<1e-12:
        return 0.

    # guess f = 0.01
    Qguess = np.copysign(np.pi*(D**2)/4 * np.sqrt(np.abs(hD)*g/(K/2 + 0.02*L/D)),hD)
    def eq(x):
        Q=x[0]
        return [hD - headDrop(Q, L, D, eD, K)]

    return root(eq, [Qguess]).x[0]

In [74]:


pipes=np.array([[1, 3, 35., 0.05, 1e-6, 3.],
                [2, 4, 20., 0.05, 1e-6, 2.],
                [3, 4, 5., 0.05, 1e-6, 1.],
                [3, 5, 10., 0.05, 1e-6, 4.],
                [3, 6, 1., 0.05, 1e-6, 1.],
                [4, 5, 3., 0.05, 1e-6, 2.],
                [5, 6, 1., 0.05, 1e-6, 1.]])



#indexing starts from 0
pipe_in = pipes[:,0].astype(np.int32)-1
pipe_out = pipes[:,1].astype(np.int32)-1
lengths = pipes[:,2]
diameters = pipes[:,3]
eDs = pipes[:,4]
Kvals=pipes[:,5]

pipe_flows = np.zeros_like(lengths)
npipes = pipe_flows.size

In [109]:
z=np.array([30., 10., 0., 5., 0., 0.])
P=np.zeros_like(z)
heads = np.zeros_like(z)
#idx start from 0, nodes start from 1
external_idx=np.array([1, 2, 6])-1
internal_idx=np.array(list(set(np.arange(np.max(pipe_out)+1))-set(external_idx)))
P[external_idx]=np.array([3e5, 0., 0.])
heads[external_idx] = P[external_idx]/rho/g + z[external_idx]



In [120]:
def internal_node_flow_to_zero(internal_pressure_guesses):
    netFlows=np.zeros_like(z)
    heads[internal_idx]=internal_pressure_guesses/rho/g + z[internal_idx]
    for i in range(npipes):
        dH=heads[pipe_in[i]]- heads[pipe_out[i]]
        pipe_flows[i]=flow(dH, lengths[i], diameters[i], eDs[i], Kvals[i])
    np.add.at(netFlows, pipe_in, -pipe_flows)
    np.add.at(netFlows, pipe_out, pipe_flows)
    return netFlows[internal_idx]

In [121]:
Pint_guesses=(np.mean(heads[external_idx])-z[internal_idx])*rho*g

In [122]:
root(internal_node_flow_to_zero, Pint_guesses)

    fjac: array([[-0.76937659,  0.63652837,  0.05377074],
       [-0.47523493, -0.62659837,  0.61767811],
       [-0.4268623 , -0.44967334, -0.78459063]])
     fun: array([ 5.79991263e-12, -3.62606147e-12, -2.42334278e-12])
 message: 'The solution converged.'
    nfev: 22
     qtf: array([ 1.49311586e-10, -2.77636232e-11, -9.09307030e-12])
       r: array([ 2.75441108e-06, -2.54527005e-06,  1.44860679e-07,  5.54294532e-07,
       -5.15535962e-07,  3.06275628e-07])
  status: 1
 success: True
       x: array([ 37915.87771346, -10916.36917027,  20397.4471229 ])

In [123]:
pipe_flows

array([ 0.01870679,  0.00747239, -0.000675  ,  0.00423824,  0.01514354,
        0.0067974 ,  0.01103564])